# Homework 3 

# Задача №1 - Лес или пустыня?

Часто при анализе изображений местности необходимо понять ее характер. В частности, если определить, что на изображении преобладет вода, то имеет смысл искать корабли на таком изображении. Если на картинке густой лес, то, возможно, это не лучшая зона для посадки дрона или беспилотника.

Ваша задача - написать программу, которая будет отличать лес от пустыни. В приложении можно найти реальные спутниковые снимки лесов и пустынь.

Примеры изображений:
<table><tr>
	<td> <img src="https://i.ibb.co/nmHHctW/test_image_00.jpg" alt="Drawing" style="width: 200px;"/> </td>
	<td> <img src="https://i.ibb.co/dM77C4b/test_image_06.jpg" alt="Drawing" style="width: 200px;"/> </td>
</tr></table>

In [14]:
import os

import numpy as np
import cv2 as cv

### Reading data

In [2]:
path_images_forest_desert="lab3/desert_forest"

In [3]:
def readImagesFromDirectory(path : str, imread_mode=cv.IMREAD_COLOR) -> list[np.ndarray]:
	images=[]
	
	for file_name in os.listdir(path):
		file_name_full=f"{path}/{file_name}"
		if(os.path.isfile(file_name_full)):
			try:
				images.append(cv.imread(file_name_full, imread_mode))
			except Exception as e:
				print(e)
				pass
			
	return images

### Creaing histograms

In [4]:

def classifyImages(classes_images_base : list[list[np.ndarray]], images : list[np.ndarray]) -> list[list[float]]:
	"""Classifies images based on average histogram of base images
	
	Args:
		classes_images_base (list[list[np.ndarray]]): 
		images (list[np.ndarray]): 

	Returns:
		list[list[float]]: Class probabilities for each image
	"""
	classes_histograms=[]
	histogram_channels=[0, 1, 2]
	histogram_size=[256, 256, 256]
	histogram_range_r=[0, 256]; histogram_range_g=[0, 256]; histogram_range_b=[0, 256];
	histogram_range=histogram_range_r+histogram_range_g+histogram_range_b
	images_class_values=[]
	
	for images_base in classes_images_base:
		images_base_histograms=[]
		for image_base in images_base:
			image_base_histogram=cv.calcHist([image_base], histogram_channels, None, histogram_size, histogram_range)
			cv.normalize(image_base_histogram, image_base_histogram, 0, 1, cv.NORM_MINMAX)
			images_base_histograms.append(image_base_histogram)
		classes_histograms.append(np.average(images_base_histograms, axis=0))
	
	for image in images:
		class_values=[]
		for base_image_histogram in classes_histograms:
			class_values.append(cv.compareHist(base_image_histogram, cv.calcHist([image], histogram_channels, None, histogram_size, histogram_range), 0))
		images_class_values.append(class_values)
	
	return images_class_values

In [5]:
class_names=["forest", "desert"]
classes_images_base=[
	[cv.imread(f"{path_images_forest_desert}/test_image_00.jpg"), cv.imread(f"{path_images_forest_desert}/test_image_01.jpg")], 
	[cv.imread(f"{path_images_forest_desert}/test_image_04.jpg"), cv.imread(f"{path_images_forest_desert}/test_image_12.jpg")]
]
images=readImagesFromDirectory(path_images_forest_desert)
images_class_values=classifyImages(classes_images_base, images)
class_value_threshold=0.1

i=0
for image_class_values in images_class_values:
	class_value_max=0
	j=0
	j_class_value_max=0
	for class_value in image_class_values:
		if class_value>class_value_max:
			class_value_max=class_value
			j_class_value_max=j
		if class_value>class_value_threshold:
			print(f"Image {i} can be a {class_names[j]}.")
		j+=1
	if class_value_max>class_value_threshold:
		result=f"Image {i} is most likely a {class_names[j_class_value_max]}."
	else:
		result=f"Image {i} can't be classified."
	print(result)
	print(f"Accuracies: {image_class_values}")
	cv.imshow(result, images[i])
	cv.waitKey(0)
	cv.destroyAllWindows()
	i+=1

Image 0 can be a forest.
Image 0 is most likely a forest.
Accuracies: [0.9814483085191159, -0.06401127337196623]
Image 1 can be a forest.
Image 1 is most likely a forest.
Accuracies: [0.25325250696479124, -0.013189855398531592]
Image 2 can't be classified.
Accuracies: [0.048546379920778045, -0.018349830176498074]
Image 3 can be a forest.
Image 3 is most likely a forest.
Accuracies: [0.534970642838238, -0.06320153042086536]
Image 4 can be a desert.
Image 4 is most likely a desert.
Accuracies: [-0.054493664966895304, 0.8108632233060854]
Image 5 can be a forest.
Image 5 is most likely a forest.
Accuracies: [0.6739402559411465, -0.05347165810441724]
Image 6 can't be classified.
Accuracies: [-0.03783977642680696, 0.033170830390727636]
Image 7 can't be classified.
Accuracies: [0.009036860154759057, -0.04510664721725452]
Image 8 can be a forest.
Image 8 is most likely a forest.
Accuracies: [0.169777079154702, -0.01930765352768221]
Image 9 can be a forest.
Image 9 is most likely a forest.
Accu

# Задача №2 - Реализовать Image-blending на основе сшивки по градиентам

Задача - взять фото двух лиц : ваше и друга, с помощью метода Poisson image editing совместить глаза, нос и рот с первого изображения со вторым. Суть в том, что при использовании такого метода границы совмещенного изображения не видны.

Статья, где описан метод  

Patrick Pérez, Michel Gangnet, and Andrew Blake. 2003. Poisson image editing. ACM Trans. Graph. 22, 3 (July 2003), 313–318. https://doi.org/10.1145/882262.882269

Пример такого совмещения:

<img src="blending/blending.png" alt="Drawing" style="width: 700px;"/>


### Reading data

In [6]:
path_images_cups="lab3/cups"

In [7]:
images=readImagesFromDirectory(path_images_cups)

### Blending images

In [8]:
def blendImages(image_1 : np.ndarray, image_2 : np.ndarray) -> np.ndarray:
	images=[image_1, image_2]
	tuples_pyramides_gauss_laplace=[]
	rg=5
	
	for image in images:
		image_copy=image.copy()
		pyramid_gauss = [image_copy]
		for i in range(rg):
			image_copy=cv.pyrDown(image_copy)
			pyramid_gauss.append(image_copy)
			
		pyramid_laplace = [pyramid_gauss[rg - 1]]
		for i in range(rg - 1, 0, -1):
			upsample = cv.pyrUp(pyramid_gauss[i])
			pyramid_gauss[i-1] = np.pad(pyramid_gauss[i-1], pad_width=[(0, abs(upsample.shape[0]-pyramid_gauss[i-1].shape[0])), (0, abs(upsample.shape[1]-pyramid_gauss[i-1].shape[1])), (0, 0)], mode='edge')
			pyramid_laplace.append(cv.subtract(pyramid_gauss[i-1], upsample))
		tuples_pyramides_gauss_laplace.append((pyramid_gauss, pyramid_laplace))
	
	ls = []
	for image_1_pyramid_laplace, image_2_pyramid_laplace in zip(tuples_pyramides_gauss_laplace[0][1], tuples_pyramides_gauss_laplace[1][1]):
		ls.append(np.hstack((image_1_pyramid_laplace[:, 0:image_1_pyramid_laplace.shape[1]//2], image_2_pyramid_laplace[:, image_1_pyramid_laplace.shape[1]//2:])))
	
	image_blended=ls[0]
	for i in range(1, rg):
		image_blended=cv.pyrUp(image_blended)
		image_blended = cv.add(image_blended, ls[i])
	
	return image_blended

In [9]:
cv.imshow("Blended cup image", blendImages(images[0], images[1]))
cv.waitKey()
cv.destroyAllWindows()

# Задача №3 - Найди клетки

Даны снимки раковых клеток. Существует задача - определить стадию рака клетки по такому изображению. Для того, чтобы подойти к решению классификации рака клетки, необходимо сначала подготовить данные.

Исходные изображения в реальных задачах могут быть очень большого размера (более 20000 px). Однако из визуального анализа можно заметить, что большая часть этих снимков пустая и не несет в себе полезную информацию.

Ваша задача выделить небольшие ячейки изображений из исходного так, чтобы на ячейках было только изображение клетки.

Пример исходного изображения и нарезанных ячеек клетки.
<img src="img/cell_example.png" alt="Drawing" style="width: 500px;"/>

В качестве аргументов у функции будут значения:
1. исходное изображние;
2. размер ячейки;
3. количество ячеек.

__Доп вопрос__ - как можно выяснить какие нужны значения аргументов, чтобы они подходили для большинства исходных снимков?

### Answer

Методом проб и ошибок. Но рассмотрим бимодальное изображение (проще говоря, бимодальное изображение - это изображение, гистограмма которого имеет два пика). Для этого изображения мы можем приблизительно принять значение в середине этих пиков в качестве порогового значения, верно? Это то, что делает бинаризация Оцу. Таким образом, простыми словами, он автоматически вычисляет пороговое значение из гистограммы изображения для бимодального изображения. (Для изображений, которые не являются бимодальными, бинаризация не будет точной.)

### Reading data

In [10]:
path_images_cells="lab3/cells"

In [11]:
images=readImagesFromDirectory(path_images_cells)

### Finding cells

In [12]:
def getCells(image : np.ndarray, cell_size : tuple[int, int], cells_count : int) -> list[np.ndarray]:
    _, binary = cv.threshold(cv.cvtColor(image, cv.COLOR_BGR2GRAY), 127, 255, cv.THRESH_BINARY_INV + cv.THRESH_OTSU)
    contours, _ = cv.findContours(binary, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    images_cell = []
    for contour in contours:
        location_x, location_y, width, height = cv.boundingRect(contour)
        cell = image[location_y:location_y+height, location_x:location_x+width]
        images_cell.append(cv.resize(cell, (cell_size[0], cell_size[1])))
        if len(images_cell) >= cells_count:
            break
    
    return images_cell

In [13]:
for image in images:
	images_cell=getCells(image, (20, 20), 50)
	i=0
	for cell in images_cell:
		cv.imshow(f"Cell {i}", cell)
		i+=1
	cv.waitKey()
	cv.destroyAllWindows()